In [1]:
from utils import get_current_predictor_data,get_player_ids_for_entry
from constants import AVAILABLE_FEATURES,CREATED_FEATURES
from data_collection import DataContainer,DataProcessor
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
from scipy.optimize import linprog
import logging
import joblib
# logging.basicConfig(level=logging.DEBUG)
logging.basicConfig(level=logging.INFO)

#uncomment if you just want to see a snapshot of data
# current_stats_snap = DataGrabber.get_raw_season_stats_snap()
# players_dict,_ = extract_and_process_playerinfo_to_dict(current_stats_snap)
# players_df = process_dictdata_to_dataframe(players_dict,['id','now_cost','element_type','web_name','status']+['points_per_game','form','expected_goal_involvements','total_points','creativity','bonus','ict_index',])

In [2]:
import joblib
data_obj = DataContainer(lookback_weeks=5,range_weeks=3)
joblib.dump(data_obj, 'collected_data.pkl')

INFO:root:Starting method: get_raw_season_stats_snap
INFO:root:Starting method: query_API
INFO:root:Method query_API completed in 0.19 seconds
INFO:root:Method get_raw_season_stats_snap completed in 0.20 seconds
INFO:root:Starting method: create_and_cache_player_dict_from_snap
INFO:root:Extracting player data from dictionary of length: 555...
INFO:root:Number of players: 555
INFO:root:New player, caching for: G.Jesus
INFO:root:Starting method: query_API
INFO:root:Method query_API completed in 0.12 seconds
INFO:root:Finished fetching gameweek data for 0 out of 555 player dictionaries...
INFO:root:New player, caching for: Gabriel
INFO:root:Starting method: query_API
INFO:root:Method query_API completed in 0.13 seconds
INFO:root:New player, caching for: Havertz
INFO:root:Starting method: query_API
INFO:root:Method query_API completed in 0.12 seconds
INFO:root:New player, caching for: J.Timber
INFO:root:Starting method: query_API
INFO:root:Method query_API completed in 0.13 seconds
INFO:ro

['collected_data.pkl']

In [3]:
import joblib
data_obj = joblib.load('collected_data.pkl')
processed_data = DataProcessor(data_obj,)
joblib.dump(processed_data, 'processed_data.pkl')

/home/zhillyz/fpl/data_collection.py:59: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.players["is_injured"] = self.players["status"].apply(lambda x: 1 if x in ["i", "s", "n", "u"] else 0)
INFO:root:Starting method: get_training_data
INFO:root:Starting method: create_and_cache_player_dict_from_snap
INFO:root:Extracting player data from dictionary of length: 537...
INFO:root:Number of players: 537
INFO:root:Finished fetching gameweek data for 0 out of 537 player dictionaries...
INFO:root:Finished fetching gameweek data for 10 out of 537 player dictionaries...
INFO:root:Finished fetching gameweek data for 20 out of 537 player dictionaries...
INFO:root:Finished fetching gameweek data for 30 out of 537 player dictionaries...
INFO:root:Finished f

['processed_data.pkl']

In [4]:
import joblib
X_train,Y_train = processed_data.training_data
X_train, X_val, Y_train, y_val = train_test_split(X_train, Y_train, test_size=0.2, random_state=37)
model = make_pipeline(StandardScaler(), RandomForestRegressor(n_estimators=500))
model.fit(X_train, Y_train)

preds = model.predict(X_val)
print("MAE:", mean_absolute_error(y_val, preds))
# Save model to file
joblib.dump(model, 'fpl_model.pkl')

MAE: 1.223662509214212


['fpl_model.pkl']

In [9]:
""" Run this for best Wildcard team. No constraint on players per team"""
# Step 1: Predict Future Points for All Players
# Assuming 'model' is your trained model, and 'players_df' is a dataframe with player features
# Add predictions to your player dataframe
import joblib
processed_data = joblib.load('processed_data.pkl')
data_obj = joblib.load('collected_data.pkl')

all_data = get_current_predictor_data()
predict_data = all_data[AVAILABLE_FEATURES + CREATED_FEATURES]

# Ensure no NaN values in the features
predict_data = predict_data.fillna(0)
model = joblib.load('fpl_model.pkl')
predict_data['predicted_future_points'] = model.predict(predict_data)
predict_data.loc[predict_data['num_fixtures'] == 0, 'predicted_future_points'] = 0
predict_data[['id','element_type','web_name']] = all_data[['id','element_type','web_name']]
# Step 2: Set Constraints

# Total budget (assuming the total budget is 100.0)
budget = 100  # Total budget in FPL units

# Player positions constraints (e.g., 2 goalkeepers, 5 defenders, 5 midfielders, 3 forwards)
position_constraints = {
    1:2,
    2:5,
    3:5,
    4:3,
}

# Assuming players_df has columns: 'id', 'position', 'cost', 'predicted_future_points'
# Step 3: Optimize the Team Selection

# Define cost and predicted points as arrays for optimization
costs = predict_data['now_cost'].values / 10.0  # Convert cost from 0.1 million to actual cost (in millions)
future_points = predict_data['predicted_future_points'].values
positions = all_data['element_type'].values

# Number of players to select (e.g., 15 players for an FPL team)
num_players = 15

# Define optimization coefficients (we want to maximize future points)
c = -future_points  # We negate because linprog minimizes by default

# Constraints:
# 1. Total cost must not exceed the budget
# 2. Positional constraints (e.g., 2 goalkeepers, 5 defenders, etc.)
A_eq = []
b_eq = []
for position, max_count in position_constraints.items():
    # Create a constraint matrix for each position
    position_indices = (positions == position)
    A_eq.append(position_indices.astype(int))  # Make sure the number of players for each position doesn't exceed max_count
    b_eq.append(max_count)

# 3. Select exactly 15 players
A_eq.append(np.ones(len(predict_data)))  # Ensure we select exactly 15 players
b_eq.append(num_players)

# Step 4: Solve the optimization problem using linprog
result = linprog(c, A_eq=np.array(A_eq), b_eq=np.array(b_eq), bounds=[(0, 1)] * len(predict_data), method='highs')

# Step 5: Check for errors and Get the selected players
if result.success:
    selected_player_indices = np.where(result.x >= 0.5)[0]  # Get the players with a value >= 0.5 (i.e., selected)

    # Step 6: Get the player IDs and their details
    selected_players = predict_data.iloc[selected_player_indices]
    selected_players.sort_values(by='element_type',ascending=True,inplace=True)
    print(selected_players[['web_name', 'now_cost', 'element_type', 'predicted_future_points']])

    # Total cost of the selected players
    total_cost = selected_players['now_cost'].sum() / 10  # Convert cost from 0.1 million to actual cost
    total_points = selected_players['predicted_future_points'].sum()
    print(f"Total cost of selected players: {total_cost}")
    print(f"Total predicted points: {total_points}")
else:
    print("Optimization failed: ", result.message)
    # Debugging: Check if total cost exceeds budget or if other issues exist
    print(f"Total cost of all players: {predict_data['now_cost'].sum() / 10}")
    print(f"Number of players available: {len(predict_data)}")
    print(f"Positional constraints: {position_constraints}")


INFO:root:Starting method: get_raw_season_stats_snap
INFO:root:Starting method: query_API
INFO:root:Method query_API completed in 0.18 seconds
INFO:root:Method get_raw_season_stats_snap completed in 0.18 seconds
INFO:root:Starting method: create_and_cache_player_dict_from_snap
INFO:root:Extracting player data from dictionary of length: 555...
INFO:root:Number of players: 555
INFO:root:Finished fetching gameweek data for 0 out of 555 player dictionaries...
INFO:root:Finished fetching gameweek data for 10 out of 555 player dictionaries...
INFO:root:Finished fetching gameweek data for 20 out of 555 player dictionaries...
INFO:root:Finished fetching gameweek data for 30 out of 555 player dictionaries...
INFO:root:Finished fetching gameweek data for 40 out of 555 player dictionaries...
INFO:root:Finished fetching gameweek data for 50 out of 555 player dictionaries...
INFO:root:Finished fetching gameweek data for 60 out of 555 player dictionaries...
INFO:root:Finished fetching gameweek data 

        web_name  now_cost element_type  predicted_future_points
91       Flekken        44            1                 5.958000
536      Bentley        39            1                 6.096000
255     Robinson        48            2                 4.562000
326       Konaté        53            2                 4.902000
415        Schär        55            2                 5.082000
422         Aina        52            2                 5.981633
573   Milenković        51            2                 5.008000
99        Mbeumo        79            3                 6.640000
168         Enzo        47            3                 8.744000
182       Palmer       106            3                 6.734000
366  B.Fernandes        85            3                 6.274000
530       Souček        49            3                 7.508000
148      Welbeck        55            4                 5.476000
306        Vardy        53            4                 5.338000
447         Wood        6

/tmp/ipykernel_293320/1473402656.py:69: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  selected_players.sort_values(by='element_type',ascending=True,inplace=True)


In [10]:
current_team_ids = [383,350,88,255,366,328,402,99,321,401,58,521,71,270,295]
entry_id = 1729758#Me
# entry_id = 57510#Martin
# entry_id = 1724338#Kris
event_id = 33
transferred = False
current_team_ids = get_player_ids_for_entry(entry_id,event_id)
# if transferred:
#     current_team_ids.remove(best_transfer[0]['id'])
#     current_team_ids.append(best_transfer[1]['id'])
my_teams_predicted_points = predict_data[predict_data['id'].isin(current_team_ids)]
my_teams_predicted_points

INFO:root:Starting method: query_API
INFO:root:Method query_API completed in 0.12 seconds


,points_per_game,form,expected_goal_involvements,total_points,creativity,ict_index,cost_change_event,now_cost,bonus,value_form,horizon,num_fixtures,total_fdr,is_injured,predicted_future_points,id,element_type,web_name
58,5.1,4.2,15.97,168,313.8,203.0,0,89,29,0.5,1,0,0,0,0.0000,58,4,Watkins
725,3.0,4.6,1.67,30,31.4,30.8,0,53,3,0.9,1,0,0,0,0.0000,725,3,Malen
88,3.2,3.5,3.76,104,143.7,133.5,0,45,7,0.8,1,1,4,0,4.2000,88,2,Collins
99,6.4,7.8,19.32,210,987.5,297.6,0,79,28,1.0,1,1,4,0,6.6400,99,3,Mbeumo
255,3.1,1.0,4.49,102,678.7,170.8,-1,48,8,0.2,1,1,2,0,4.5620,255,2,Robinson
270,1.6,0.0,1.39,32,40.3,65.0,-1,38,0,0.0,1,1,4,0,1.8078,270,2,Greaves
295,1.6,0.5,2.12,50,224.7,93.6,0,41,2,0.1,1,1,2,0,1.5162,295,2,Justin
321,3.3,1.0,9.15,100,435.5,148.0,0,72,15,0.1,1,1,3,0,4.7760,321,4,Gakpo
328,9.7,3.5,31.11,320,1037.3,431.4,0,138,51,0.3,1,1,3,0,6.1500,328,3,M.Salah
350,4.0,4.4,6.96,128,466.6,166.0,1,63,11,0.7,1,0,0,0,0.0000,350,2,Gvardiol


In [7]:
data_obj.id_to_name_map[89]

AttributeError: 'DataContainer' object has no attribute 'id_to_name_map'

In [ ]:
"""Run this if you want to know the best 1 to 1 transfer to make"""
current_team_ids = [383,350,88,255,366,328,402,99,321,401,58,521,71,270,295]  # your 15-player team as player IDs
budget_left = 0.8        # remaining budget from original 1000 (or 100.0)

# Get your full player pool with predicted future points
all_players = predict_data.copy()
# Get your current team details
current_team = all_players[all_players['id'].isin(current_team_ids)]

best_transfer = None
best_gain = 0

# Loop through all current team players
for i, old_player in current_team.iterrows():
    old_pos = old_player['element_type']
    old_cost = old_player['now_cost']
    
    # Look at all possible replacements in same position
    candidates = all_players[
        (all_players['element_type'] == old_pos) &
        (~all_players['id'].isin(current_team_ids))  # not already owned
    ]
    
    for j, new_player in candidates.iterrows():
        cost_diff = new_player['now_cost'] - old_cost
        if cost_diff <= budget_left:
            # Create new team with the candidate transfer
            temp_team = current_team.copy()
            temp_team = temp_team[temp_team['id'] != old_player['id']]
            temp_team = pd.concat([temp_team, new_player.to_frame().T])
            
            team_points = temp_team['predicted_future_points'].sum()
            
            # Track the best one
            if team_points > best_gain:
                best_gain = team_points
                best_transfer = (old_player, new_player)
from pprint import pprint
pprint(best_transfer)

(points_per_game                      4.8
form                                 8.4
expected_goal_involvements         10.71
total_points                         143
creativity                         520.1
ict_index                          163.8
chance_of_playing_next_round         100
now_cost                              52
bps                                  526
horizon                                1
num_fixtures                           1
total_fdr                              3
is_injured                             0
predicted_future_points             3.04
id                                   402
element_type                           3
web_name                        J.Murphy
Name: 402, dtype: object,
 points_per_game                    3.5
form                               0.8
expected_goal_involvements        6.42
total_points                       104
creativity                       219.5
ict_index                        125.6
chance_of_playing_next_round       100
no

In [ ]:
data_obj.players[3]["status"]